In [1]:
import efficientnet

In [2]:
import os 
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import cv2
import imageio
import skimage.transform as skTrans
from nilearn.image import resample_img
import nibabel as nib
from tqdm import tqdm
from PIL import Image
from ipywidgets import *

In [4]:
import fastai; fastai.__version__
from fastai.basics import *
from fastai.vision.all import *
from fastai.data.transforms import *

In [5]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    #array   = np.array(array)
    array   = np.rot90(np.array(array))
    return(array)

In [6]:
# Preprocess the nii file 
# Source https://docs.fast.ai/medical.imaging

dicom_windows = types.SimpleNamespace(
    brain=(80,40),
    subdural=(254,100),
    stroke=(8,32),
    brain_bone=(2800,600),
    brain_soft=(375,40),
    lungs=(1500,-600),
    mediastinum=(350,50),
    abdomen_soft=(400,50),
    liver=(150,30),
    spine_soft=(250,50),
    spine_bone=(1800,400),
    custom = (200,60)
)

@patch
def windowed(self:Tensor, w, l):
    px = self.clone()
    px_min = l - w//2
    px_max = l + w//2
    px[px<px_min] = px_min
    px[px>px_max] = px_max
    return (px-px_min) / (px_max-px_min)

In [7]:
# Preprocessing functions
# Source https://docs.fast.ai/medical.imaging

class TensorCTScan(TensorImageBW): _show_args = {'cmap':'bone'}

@patch
def freqhist_bins(self:Tensor, n_bins=100):
    "A function to split the range of pixel values into groups, such that each group has around the same number of pixels"
    imsd = self.view(-1).sort()[0]
    t = torch.cat([tensor([0.001]),
                   torch.arange(n_bins).float()/n_bins+(1/2/n_bins),
                   tensor([0.999])])
    t = (len(imsd)*t).long()
    return imsd[t].unique()
    
@patch
def hist_scaled(self:Tensor, brks=None):
    "Scales a tensor using `freqhist_bins` to values between 0 and 1"
    if self.device.type=='cuda': return self.hist_scaled_pt(brks)
    if brks is None: brks = self.freqhist_bins()
    ys = np.linspace(0., 1., len(brks))
    x = self.numpy().flatten()
    x = np.interp(x, brks.numpy(), ys)
    return tensor(x).reshape(self.shape).clamp(0.,1.)
    
    
@patch
def to_nchan(x:Tensor, wins, bins=None):
    res = [x.windowed(*win) for win in wins]
    if not isinstance(bins,int) or bins!=0: res.append(x.hist_scaled(bins).clamp(0,1))
    dim = [0,1][x.dim()==3]
    return TensorCTScan(torch.stack(res, dim=dim))

In [9]:
Size_X =128
Size_Y =128
n_classes = 2

In [10]:
@patch
def save_jpg(x:(Tensor), path, wins, bins=None, quality=90):
    fn = Path(path).with_suffix('.jpg')
    x = (x.to_nchan(wins, bins)*255).byte()
    im = Image.fromarray(x.permute(1,2,0).numpy(), mode=['RGB','CMYK'][x.shape[0]==4])
    #im=im.convert('BGR')
    im = im.resize((Size_Y,Size_X))
    im = im.rotate(angle = 270)
    im= np.array(im)
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    return im
    #im.save(fn, quality=quality)

In [11]:
#from keras.utils import normalize
import tensorflow as tf
import segmentation_models as sm
BACKBONE = 'efficientnetb4'
preprocess_input = sm.get_preprocessing(BACKBONE)

2022-06-14 19:55:11.944594: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Segmentation Models: using `tf.keras` framework.


In [12]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

In [18]:
# define model
#model = sm.PSPNet(BACKBONE, encoder_weights = None, classes =3 ,encoder_freeze=False, activation='sigmoid',input_shape=(Size_X,Size_Y,3), downsample_factor=8,psp_conv_filters=512, psp_pooling_type='avg',psp_use_batchnorm=True,psp_dropout=0.2)
model = sm.Unet(BACKBONE, encoder_weights = 'imagenet', classes =3 ,encoder_freeze=False,input_shape=(128,128,3),activation='softmax')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),loss=sm.losses.cce_jaccard_loss, metrics=[sm.metrics.IOUScore(class_indexes=2,per_image=True),sm.metrics.FScore(class_indexes=2,per_image=True),sm.metrics.Precision(class_indexes=2),sm.metrics.Recall(class_indexes=2)],)
print(model.summary())


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 64, 48)   1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 64, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 64, 48)   0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [11]:
# define model
model = sm.PSPNet(BACKBONE, encoder_weights = None, classes =3 ,encoder_freeze=False, activation='softmax',input_shape=(Size_Y,Size_X,3), downsample_factor=8,psp_conv_filters=512, psp_pooling_type='avg',psp_use_batchnorm=True,psp_dropout=0.2)
model.compile('Adam', loss=sm.losses.cce_jaccard_loss, metrics=[sm.metrics.IOUScore(class_indexes=2),sm.metrics.FScore(class_indexes=2),sm.metrics.Precision(class_indexes=2),sm.metrics.Recall(class_indexes=2)])
print(model.metrics_names)

2022-05-28 17:27:17.881383: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-28 17:27:17.882818: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-28 17:27:17.884094: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-28 17:27:17.885281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 2 with properties: 
pciBusID: 0000:81:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2

[]


In [19]:
model.load_weights('weights/last/effb4_unet240.h5')

In [20]:
vol_list = []
mask_list = []
for filename in sorted(os.listdir('new/val/vol')):
    vol_list.append(filename)
for filename in sorted(os.listdir('new/val/mask')):
    mask_list.append(filename)

df_files = pd.DataFrame(vol_list,columns=['volname'])
df_files['maskname'] = mask_list
#df_files.sort_values(by=['filename'], ascending=True)

In [21]:
df_files

,volname,maskname
0,volume-107.nii,segmentation-107.nii
1,volume-108.nii,segmentation-108.nii
2,volume-109.nii,segmentation-109.nii
3,volume-110.nii,segmentation-110.nii
4,volume-124.nii,segmentation-124.nii
5,volume-125.nii,segmentation-125.nii
6,volume-126.nii,segmentation-126.nii
7,volume-127.nii,segmentation-127.nii
8,volume-22.nii,segmentation-22.nii
9,volume-23.nii,segmentation-23.nii


In [23]:
from tensorflow.keras.utils import to_categorical
dice=[]
rows = len(df_files)
dir_path = 'new/val/'
for i in range(0,rows):
  vol_path=os.path.join(dir_path+'vol/'+df_files['volname'][i])
  mask_path=os.path.join(dir_path+'mask/'+df_files['maskname'][i])
  curr_ct = read_nii(vol_path)
  curr_mask = read_nii(mask_path)
  curr_dim = curr_mask.shape[2]
  test_img=[]
  test_mask=[]
  for curr_slice in range(0,curr_dim):
    data = tensor(curr_ct[...,curr_slice].astype(np.float32))
    img = data.save_jpg(f"none.jpg", [dicom_windows.liver,dicom_windows.custom])
    img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    test_img.append(img)        
    mask = Image.fromarray(curr_mask[...,curr_slice].astype('uint8'), mode="L")
    mask = mask.resize((Size_Y,Size_X))
    mask = mask.rotate(angle=270)
    mask = np.array(mask)
    test_mask.append(mask)
  test_img = np.array(test_img)
  test_mask = np.array(test_mask)
  U = test_img
  V = test_mask
  V = np.expand_dims(V, axis=3)
  test_masks_cat = to_categorical(V)
  v_train_cat = test_masks_cat.reshape((V.shape[0], V.shape[1], V.shape[2], 3))
  results = model.evaluate(U,test_masks_cat)
  dice.append(results)
  print(results)
#print(dice)

25/25 [==============================] - 1s 37ms/step - loss: 1.2011 - iou_score: 2.1106e-08 - f1-score: 2.1106e-08 - precision: 8.9561e-10 - recall: 0.8400
[1.2010539770126343, 2.110598273930009e-08, 2.110598273930009e-08, 8.956134167625862e-10, 0.8399999737739563]
27/27 [==============================] - 1s 36ms/step - loss: 1.2458 - iou_score: 1.9498e-08 - f1-score: 1.9498e-08 - precision: 6.3742e-10 - recall: 0.7037
[1.2457945346832275, 1.949805827905493e-08, 1.949805827905493e-08, 6.374174521539544e-10, 0.703703761100769]


KeyboardInterrupt: 

In [34]:
dice=np.array(dice)
dice.mean(axis=0)
#dice

array([0.29552502, 0.79284553, 0.79404802, 0.99547175, 0.79704482])

# Model testing for submission

In [13]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    #array   = np.array(array)
    array   = np.rot90(np.array(array))
    return(array, ct_scan)

In [14]:
@patch
def save_jpg(x:(Tensor), path, wins, bins=None, quality=90):
    fn = Path(path).with_suffix('.jpg')
    x = (x.to_nchan(wins, bins)*255).byte()
    im = Image.fromarray(x.permute(1,2,0).numpy(), mode=['RGB','CMYK'][x.shape[0]==4])
    #im=im.convert('BGR')
    #im = ImageOps.mirror(im.rotate(angle = 180))
    im = im.rotate(angle = -90)
    im = im.resize((Size_Y,Size_X))
    im= np.array(im)
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    return im
    #im.save(fn, quality=quality)

In [15]:
def togray(mask_gen):
  mask_gen[mask_gen[:,:,0]>0.5,0]=0 
  mask_gen[mask_gen[:,:,1]>0.5,1]=1
  mask_gen[mask_gen[:,:,2]>0.5,2]=2
  res=mask_gen[:,:,0]+mask_gen[:,:,1]+mask_gen[:,:,2]
  res=res.astype(np.uint8)
  res = cv2.resize(res, (512, 512))
  return res

def gray2(mask):
    res=np.zeros((mask.shape[0],mask.shape[1]))
    for i in range (0,mask.shape[0]):
        for j in range (0,mask.shape[1]):
            l=np.argmax([mask[i,j,0],mask[i,j,1],mask[i,j,2]])
            res[i,j]=l
    res=res.astype(np.uint8)
    res = cv2.resize(res, (512, 512))
    return res

In [18]:
vol_list = []
mask_list = []
for filename in sorted(os.listdir('new/test/')):
    vol_list.append(filename)
df_files = pd.DataFrame(vol_list,columns=['volname'])

print(df_files.head(20))

               volname
0    test-volume-0.nii
1    test-volume-1.nii
2   test-volume-10.nii
3   test-volume-11.nii
4   test-volume-12.nii
5   test-volume-13.nii
6   test-volume-14.nii
7   test-volume-15.nii
8   test-volume-16.nii
9   test-volume-17.nii
10  test-volume-18.nii
11  test-volume-19.nii
12   test-volume-2.nii
13  test-volume-20.nii
14  test-volume-21.nii
15  test-volume-22.nii
16  test-volume-23.nii
17  test-volume-24.nii
18  test-volume-25.nii
19  test-volume-26.nii


In [24]:
rows = len(df_files)
dir_path = 'new/test/'
for i in range(0,rows):
  vol_path=os.path.join(dir_path+df_files['volname'][i])
  curr_ct, nifimg = read_nii(vol_path)
  curr_dim = curr_ct.shape[2]
  res_mask=np.zeros(curr_ct.shape)
    
  file_name = str(df_files.loc[i,'volname']).split('-')[2]
  number = str(file_name).split('.')[0]
    
  for curr_slice in range(0,curr_dim):
    data = tensor(curr_ct[...,curr_slice].astype(np.float32))
    img = data.save_jpg(f"new/test/test_frames/_slice_{curr_slice}.jpg", [dicom_windows.liver,dicom_windows.custom])     
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    mas=pred.reshape((Size_Y,Size_X,3))
    mas = gray2(mas)
    res_mask[...,curr_slice]=mas
  res_mask = np.array(res_mask)
  res_mask = res_mask.astype(np.uint8)
  clip = nib.Nifti1Image(res_mask, nifimg.affine, nifimg.header)
  clip.set_data_dtype(np.uint8)
  nib.save(clip, f"new/res/test-segmentation-{number}.nii")
  print(np.unique(res_mask))
  print('completed'+str(i))

[0 1 2]
completed0
[0 1 2]
completed1
[0 1 2]
completed2
[0 1 2]
completed3
[0 1 2]
completed4
[0 1 2]
completed5
[0 1 2]
completed6
[0 1 2]
completed7
[0 1 2]
completed8
[0 1 2]
completed9
[0 1 2]
completed10
[0 1 2]
completed11
[0 1 2]
completed12
[0 1 2]
completed13
[0 1 2]
completed14
[0 1 2]
completed15
[0 1 2]
completed16
[0 1 2]
completed17
[0 1 2]
completed18
[0 1 2]
completed19
[0 1 2]
completed20
[0 1 2]
completed21
[0 1 2]
completed22
[0 1 2]
completed23
[0 1 2]
completed24
[0 1 2]
completed25
[0 1 2]
completed26
[0 1 2]
completed27
[0 1 2]
completed28
[0 1 2]
completed29
[0 1 2]
completed30
[0 1 2]
completed31
[0 1 2]
completed32
[0 1 2]
completed33
[0 1 2]
completed34
[0 1 2]
completed35
[0 1 2]
completed36
[0 1 2]
completed37
[0 1 2]
completed38
[0 1 2]
completed39
[0 1 2]
completed40
[0 1 2]
completed41
[0 1 2]
completed42
[0 1 2]
completed43
[0 1 2]
completed44
[0 1 2]
completed45
[0 1 2]
completed46
[0 1 2]
completed47
[0 1 2]
completed48
[0 1 2]
completed49
[0 1 2]
co

no tumor detected in these numbers 0,2,3,4,6,21,24,25,27,28,32,33,35,36,38,42,47,48,56,60,62,64,65,69